In [ ]:
import telebot
import asyncio
from googletrans import Translator

API_TOKEN = '7801269731:AAEdoWKq2sMRyq2VsI2GoXDv4e6EXO6IFKc'

bot = telebot.TeleBot(API_TOKEN)
translator = Translator()

def sync_translate(text, dest='ru'):
    """
    Синхронно выполняет асинхронную функцию перевода.
    """
    return asyncio.run(translator.translate(text, dest=dest))

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    welcome_text = (
        "Привет!\n"
        "Я бот для перевода сообщений.\n\n"
        "Также я перевожу сообщения в каналах с любого языка, если они не на русском."
    )
    bot.reply_to(message, welcome_text)

@bot.message_handler(content_types=['text'])
def handle_private_and_group(message):
    chat_type = message.chat.type
    if chat_type in ['group', 'supergroup']:
        if message.text:
            try:
                translation = sync_translate(message.text, dest='ru')
                if translation.src.lower() != 'ru':
                    bot.reply_to(message, f"Перевод с {translation.src}:\n{translation.text}")
            except Exception as e:
                bot.reply_to(message, f"Ошибка перевода: {e}")
    elif chat_type == 'private':
        if message.forward_date:
            if message.text:
                try:
                    translation = sync_translate(message.text, dest='ru')
                    bot.reply_to(message, f"Перевод:\n{translation.text}")
                except Exception as e:
                    bot.reply_to(message, f"Ошибка перевода: {e}")
            else:
                bot.reply_to(message, "Сообщение не содержит текста для перевода.")
        else:
            bot.reply_to(message, "Пожалуйста, перешлите мне сообщение для перевода.")

@bot.channel_post_handler(content_types=['text'])
def handle_channel_post(message):
    if message.text:
        try:
            translation = sync_translate(message.text, dest='ru')
            if translation.src.lower() != 'ru':
                bot.send_message(message.chat.id, f"Перевод с {translation.src}:\n{translation.text}")
        except Exception as e:
            bot.send_message(message.chat.id, f"Ошибка перевода: {e}")

if __name__ == '__main__':
    bot.polling(none_stop=True)
